In [82]:
%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [83]:
#create a code that using selenium opens a browser and goes to starplus.com

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
import os
from dotenv import load_dotenv

load_dotenv()

user_name = os.getenv("STARPLUS_USERNAME")
password = os.getenv("STARPLUS_PASSWORD")

driver = webdriver.Chrome()
driver.get("https://www.starplus.com")

wait = WebDriverWait(driver, 20)

#click on the login button



In [84]:
#login

path = './/a[@href="/login"]'
login_button = driver.find_element(By.XPATH, path)
login_button.click()

time.sleep(10)

In [85]:
#enter email
path = './/input[@id="email"]'
email_field = driver.find_element(By.XPATH, path)
email_field.send_keys(f"{user_name}")

In [86]:
#click on continuar
path = './/button[@aria-label="Aceptar y continuar"]'
continue_button = driver.find_element(By.XPATH, path)
continue_button.click()

wait.until(EC.presence_of_element_located((By.XPATH, './/input[@aria-label="Escribir contraseña"]')))

<selenium.webdriver.remote.webelement.WebElement (session="e36eb645386c0ea6b1227926fbfe302e", element="EC77B31682574C5990162895B0F7BF3A_element_20")>

In [87]:
#enter password
path = './/input[@aria-label="Escribir contraseña"]'
password_field = driver.find_element(By.XPATH, path)
password_field.send_keys(f"{password}")

In [88]:
#click on inicar sesion
path = './/button[@aria-label="INICIAR SESIÓN"]'
login_button = driver.find_element(By.XPATH, path)
login_button.click()
user = "Jose"
wait.until(EC.presence_of_element_located((By.XPATH, f'.//h3[text()="{user}"]')))

<selenium.webdriver.remote.webelement.WebElement (session="e36eb645386c0ea6b1227926fbfe302e", element="EC77B31682574C5990162895B0F7BF3A_element_32")>

In [89]:
#busca el h3 que tiene texto "Jose"

user = "Jose"
path = f'.//h3[text()="{user}"]'
user = driver.find_element(By.XPATH, path)
user.click()

wait.until(EC.presence_of_element_located((By.XPATH, './/a[@aria-label="BÚSQUEDA"]')))

<selenium.webdriver.remote.webelement.WebElement (session="e36eb645386c0ea6b1227926fbfe302e", element="EC77B31682574C5990162895B0F7BF3A_element_39")>

In [90]:
#Encuentra el botn de busqueda
path = './/a[@aria-label="BÚSQUEDA"]'
search_button = driver.find_element(By.XPATH, path)
search_button.click()

wait.until(EC.presence_of_element_located((By.XPATH, './/input[@aria-label="Título o equipo"]')))

<selenium.webdriver.remote.webelement.WebElement (session="e36eb645386c0ea6b1227926fbfe302e", element="EC77B31682574C5990162895B0F7BF3A_element_44")>

In [91]:
#busca la serie
serie = "How I met your mother"
path = './/input[@aria-label="Título o equipo"]'
serie_field = driver.find_element(By.XPATH, path)
serie_field.send_keys(f"{serie}")

time.sleep(10)

In [93]:
#get results
path = './/div[@aria-label="Search results"]/div[1]'
results = driver.find_element(By.XPATH, path)
results.click()

wait.until(EC.presence_of_element_located((By.XPATH, './/div[@class="slick-track"]')))

<selenium.webdriver.remote.webelement.WebElement (session="e36eb645386c0ea6b1227926fbfe302e", element="EC77B31682574C5990162895B0F7BF3A_element_83")>

In [94]:
#seasons
path = './/div[@class="slick-track"]'
seasons = driver.find_element(By.XPATH, path)
seasons_range =  range(1, len(seasons.text.split("\n")) + 1)

serie = {}

for season in seasons_range:
    temp = f'.//div[@class="slick-track"]/div[{season}]/div[1]'
    season_button = driver.find_element(By.XPATH, temp)
    season_button.click()
    time.sleep(5)

    path = f'.//div[@data-gv2containerkey="details_episodes"]/div/div'
    episodes = driver.find_element(By.XPATH, path)

    current_episode = 0

    episodes = {}

    while True:
        current_episode += 1
        temp = f'.//div[@data-gv2containerkey="details_episodes"]/div/div/div/div/div[{current_episode}]/div/div/div/a'
        temp_name = f'.//div[@data-gv2containerkey="details_episodes"]/div/div/div/div/div[{current_episode}]/div'
        try:
            link = driver.find_element(By.XPATH, temp).get_attribute("data-gv2elementvalue")
            chapter_name = driver.find_element(By.XPATH, temp_name).text
        except:
            serie[f"season_{season}"] = episodes
            break

        if chapter_name != "":
            episodes[f"episode_{current_episode}"] = {"link": "https://www.starplus.com/es-419/video/"+link, 
                                                      "chapter_name": chapter_name}
        
        else:
            try:
                #get button to load more
                path = './/button[@data-testid="arrow-right"]'
                load_more = driver.find_element(By.XPATH, path)
                load_more.click()
                time.sleep(2)
                continue
            except:
                pass


In [95]:
#save series to json
import json

with open('data.json', 'w') as fp:
    json.dump(serie, fp)
    

In [ ]:
#I want to add to each link "https://www.starplus.com/es-419/video/"

for season in serie:
    for link in serie[season]["links"]:
        link = "https://www.starplus.com/es-419/video/"+link
        serie[season]["links"] = link


serie
        

{'season_1': {'links': 'https://www.starplus.com/es-419/video/ea314cde-c1bd-4926-b635-138cf8be4d53',
  'chapter_names': ['1. Piloto (23 min)\nTed comienza a contarles a sus hijos la historia de cómo conoció a su madre.',
   '2. Jirafa púrpura (23 min)\nTed organiza una fiesta con la esperanza de ver de nuevo a Robin.',
   '3. El dulce sabor de la libertad (22 min)\nLos agentes de seguridad del aeropuerto detienen a Barney y a Ted.',
   '4. El regreso de la camiseta (23 min)\nTed se reencuentra con un viejo amor.',
   '5. ¡Okey, genial! (23 min)\nRobin, Ted y Barney van a un nuevo club nocturno.',
   '7. Agencia matrimonial (23 min)\nTed contrata una agencia matrimonial.',
   '8. El duelo (23 min)\nLily se muda oficialmente con Marshall y Ted.',
   '9. Atracón de pavo (23 min)\nRobin, Barney y Ted son voluntarios en un refugio para gente sin hogar en el Día Acción de Gracias.',
   '11. La limusina (23 min)\nLa víspera de Año Nuevo se descontrola. Ted alquila una limusina para ir de fies